In [ ]:
%load_ext autoreload
%autoreload 2
from oxeo.water.datamodules.datasets import IterableTileDataset
import oxeo.water.datamodules.transforms as oxeotransforms 
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch
import matplotlib.pyplot as plt
from skimage import img_as_float
from skimage.exposure import equalize_adapthist, rescale_intensity
import numpy as np
from oxeo.water.models.utils import load_tile, TilePath, tile_from_id

In [ ]:
def plot_imgs_in_row(imgs, labels=["img", "water", "clouds"], figsize=(8,5)):
    rows = 1
    cols = len(imgs)
    fig, ax = plt.subplots(rows, cols, figsize=figsize)
    # axes are in a two-dimensional array, indexed by [row, col]
   # fig.tight_layout(pad=0.4, w_pad=0.5, h_pad=1.0)

    for i in range(cols):
        ax[i].set_title(labels[i])
        img = imgs[i]
        ax[i].imshow(img, vmin=0.0, vmax=1.0, interpolation=None)  
        ax[i].axis('off')
    fig

In [ ]:
tiles = []
for t in ["43_P_10000_65_137","43_P_10000_65_138"]:
    for c in ["landsat-5", "landsat-7", "landsat-8", "sentinel-2"]:

        tiles.append(TilePath(tile_from_id(t), c))
ds = IterableTileDataset(tiles,
                         transform=oxeotransforms.FilterZeros(["pekel"],0.99),
                         masks=("pekel","cloud_mask"),
                         target_size=1000, chip_size=256, 
                         bands=["nir", "red", "green", "blue", "swir1", "swir2"], 
                         revisits_per_epoch=500,
                         samples_per_revisit=10020)


    

In [ ]:
def worker_init_fn(worker_id):
    """Configures each dataset worker process.

    Just has one job!  To call SatelliteDataset.per_worker_init().
    """
    # get_worker_info() returns information specific to each worker process.
    worker_info = torch.utils.data.get_worker_info()
    if worker_info is None:
        print("worker_info is None!")
    else:
        dataset_obj = worker_info.dataset  # The Dataset copy in this worker process.
        dataset_obj.per_worker_init()

        

def notnone_collate_fn(batch):
    len_batch = len(batch)  # original batch length
    batch = list(filter(lambda x: x is not None, batch))  # filter out all the Nones
    if len_batch > len(
        batch
    ):  # if there are samples missing just use existing members, doesn't work if you reject every sample in a batch
        diff = len_batch - len(batch)
        for _ in range(diff):
            batch = batch + batch[:diff]
    return torch.utils.data.dataloader.default_collate(batch)

dataloader = DataLoader(ds, batch_size=32,  
                        num_workers=4, 
                        worker_init_fn=worker_init_fn, collate_fn=notnone_collate_fn)

In [ ]:
def plot_bands(arr, band_names=["nir", "red", "green", "blue", "swir1", "swir2"]):
    cols = arr.shape[0]
    
    rows = 1
    fig, ax = plt.subplots(rows, cols, figsize=(15,8))
    # axes are in a two-dimensional array, indexed by [row, col]
   # fig.tight_layout(pad=0.4, w_pad=0.5, h_pad=1.0)
    for i in range(cols):
        ax[i].set_title('Band ' + band_names[i])
        img = arr[i]
        vmin, vmax = np.percentile(img, q=(0, 99))
        img = rescale_intensity(img,in_range=(vmin,vmax))

        ax[i].imshow(img)  
        ax[i].axis('off')
    fig

In [ ]:
for sample in dataloader:
    img = sample["image"].numpy()
    img = img[0]
    plot_bands(img)
    plt.show()

In [ ]:

for sample in dataloader:
    img = sample["image"][0][[1,2,3]].numpy().transpose(1,2,0)
    pekel = sample["pekel"].numpy()[0].squeeze()
    
    mask = sample["cloud_mask"].numpy()[0].squeeze()
    #mask[0,0] = 1.0
    #pekel[0,0]=1.0
    print("pekel min", pekel.min(), "coud_min", mask.min())
    vmin, vmax = np.percentile(img, q=(2, 98))
    img = rescale_intensity(img,in_range=(vmin,vmax), out_range=(0,1))
    plot_imgs_in_row([img, pekel, mask], labels=[f"{sample['constellation'][0]}_img", 
                                                 f"{sample['constellation'][0]}_water", 
                                                 f"{sample['constellation'][0]}_cloud"])
    plt.show()